# DITL with ADCS physics

This notebook simulates multiple slews driven by the ACS and plots reaction-wheel stored momentum over time.


In [ ]:
# ruff: noqa: E402
import math
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rust_ephem import TLEEphemeris
from tqdm import tqdm

from conops import MissionConfig, Queue, QueueDITL
from conops.common import ACSMode
from conops.config.visualization import VisualizationConfig
from conops.visualization import plot_ditl_telemetry

# Shared ephemeris and targets
begin = datetime(2025, 1, 1)
end = begin + timedelta(hours=12)
ephem = TLEEphemeris(tle="../example.tle", begin=begin, end=end, step_size=10)


number_of_targets = 1000
np.random.seed(43)
target_ra = np.random.uniform(0, 360, number_of_targets)
target_dec = np.random.uniform(-90, 90, number_of_targets)
targids = list(range(10000, 10000 + number_of_targets))


def make_base_config(mtq_dipole: float) -> MissionConfig:
    cfg_local = MissionConfig.from_json_file("../example_config.json")
    acs = cfg_local.spacecraft_bus.attitude_control
    acs.spacecraft_moi = (45.0, 45.0, 45.0)
    acs.wheel_enabled = False
    acs.magnetorquers = [
        {
            "name": "mtq_x",
            "orientation": (1, 0, 0),
            "dipole_strength": mtq_dipole,
            "power_draw": 5.0,
        },
        {
            "name": "mtq_y",
            "orientation": (0, 1, 0),
            "dipole_strength": mtq_dipole,
            "power_draw": 5.0,
        },
        {
            "name": "mtq_z",
            "orientation": (0, 0, 1),
            "dipole_strength": mtq_dipole,
            "power_draw": 5.0,
        },
    ]
    acs.magnetorquer_bfield_T = 3e-5
    acs.cp_offset_body = (0.25, 0.0, 0.0)
    acs.residual_magnetic_moment = (0.05, 0, 0)
    acs.drag_area_m2 = 1.5
    acs.drag_coeff = 2.2
    acs.solar_area_m2 = 1.3
    acs.solar_reflectivity = 1.3
    acs.use_msis_density = True
    cfg_local.constraint.ephem = ephem

    if cfg_local.ground_stations is not None:
        for station in cfg_local.ground_stations.stations:
            station.schedule_probability = 1.0

    return cfg_local


def build_queue(cfg_local: MissionConfig) -> Queue:
    q = Queue(ephem=ephem, config=cfg_local)
    q.slew_distance_weight = 0.0
    for i in tqdm(range(number_of_targets), leave=False):
        q.add(
            merit=40,
            ra=target_ra[i],
            dec=target_dec[i],
            obsid=targids[i],
            name=f"pointing_{targids[i]}",
            exptime=1000,
            ss_min=300,
        )
    return q


def run_case(
    wheel_name: str, max_mom: float, max_torque: float, mtq_dipole: float
) -> tuple[QueueDITL, dict]:
    cfg_case = make_base_config(mtq_dipole)
    acs_case = cfg_case.spacecraft_bus.attitude_control

    s = math.sqrt(2 / 3)
    c = math.sqrt(1 / 3)
    acs_case.wheels = [
        {
            "name": "rw1",
            "orientation": (+s, 0.0, +c),
            "max_torque": max_torque,
            "max_momentum": max_mom,
        },
        {
            "name": "rw2",
            "orientation": (-s, 0.0, +c),
            "max_torque": max_torque,
            "max_momentum": max_mom,
        },
        {
            "name": "rw3",
            "orientation": (0.0, +s, -c),
            "max_torque": max_torque,
            "max_momentum": max_mom,
        },
        {
            "name": "rw4",
            "orientation": (0.0, -s, -c),
            "max_torque": max_torque,
            "max_momentum": max_mom,
        },
    ]

    queue_case = build_queue(cfg_case)
    # Validation warnings are printed automatically when QueueDITL creates the ACS
    ditl_case = QueueDITL(
        config=cfg_case, ephem=ephem, begin=begin, end=end, queue=queue_case
    )
    ditl_case.acs._wheel_mom_margin = 1.0  # disable margin for what-if runs
    ditl_case.step_size = 10
    ditl_case.calc()

    mode_arr = np.array(ditl_case.mode)
    science_frac = float(np.mean(mode_arr == ACSMode.SCIENCE) * 100)
    slew_frac = float(np.mean(mode_arr == ACSMode.SLEWING) * 100)
    desat_events = getattr(ditl_case.acs, "desat_events", 0)
    wm = np.array(getattr(ditl_case, "wheel_momentum_fraction", []))
    wm_raw = np.array(getattr(ditl_case, "wheel_momentum_fraction_raw", []))
    wt = np.array(getattr(ditl_case, "wheel_torque_fraction", []))
    per_wheel = getattr(ditl_case, "wheel_per_wheel_max_raw", {}) or {
        getattr(w, "name", f"rw{i}"): abs(w.current_momentum) / w.max_momentum
        for i, w in enumerate(ditl_case.acs.reaction_wheels)
    }
    raw_peak = max(per_wheel.values()) if per_wheel else 0.0

    summary = {
        "name": wheel_name,
        "mtq_dipole": mtq_dipole,
        "max_mom": max_mom,
        "max_torque": max_torque,
        "science_%": science_frac,
        "slew_%": slew_frac,
        "desat_events": desat_events,
        "max_mom_frac": float(wm.max()) if wm.size else 0.0,
        "max_mom_frac_raw": float(wm_raw.max()) if wm_raw.size else 0.0,
        "p95_mom_frac": float(np.quantile(wm, 0.95)) if wm.size else 0.0,
        "p95_mom_frac_raw": float(np.quantile(wm_raw, 0.95)) if wm_raw.size else 0.0,
        "raw_peak_per_wheel": raw_peak,
        "per_wheel_peaks": per_wheel,
        "max_torque_frac": float(wt.max()) if wt.size else 0.0,
    }
    return ditl_case, summary

In [ ]:
# Single MTQ + wheel configuration
single_wheel = ("1.0Nms_0.06Nm", 1.0, 0.06)
single_mtq = 32  # A·m^2

ditl_case, summary_single = run_case(*single_wheel, single_mtq)
print(summary_single)

fig, axes = plot_ditl_telemetry(
    ditl_case,
    figsize=(12, 8),
    config=VisualizationConfig(font_family="DejaVu Sans"),
)
fig.suptitle(f"Case: {summary_single['name']} mtq={single_mtq}")
plt.show()

In [ ]:
import numpy as np

mode = np.array(ditl_case.mode)
step = float(ditl_case.step_size)

# Pull telemetry (safe defaults)
wm_raw = np.array(getattr(ditl_case, "wheel_momentum_fraction_raw", []), dtype=float)
pass_rate = np.array(getattr(ditl_case, "pass_tracking_rate_deg_s", []), dtype=float)
pass_t_target = np.array(getattr(ditl_case, "pass_torque_target_mag", []), dtype=float)

# PASS mask
is_pass = mode == ACSMode.PASS
pass_n = int(is_pass.sum())

# PASS torque sanity (expected vs target)
moi_cfg = np.array(
    ditl_case.config.spacecraft_bus.attitude_control.spacecraft_moi, dtype=float
)
i_axis = float(np.mean(moi_cfg))  # aligned with current PASS model
if pass_rate.size == len(mode) and pass_n > 0:
    last_rate = np.roll(pass_rate, 1)
    last_rate[0] = 0.0
    accel_req = (pass_rate - last_rate) / step  # deg/s^2
    t_expected = np.abs(i_axis * accel_req * (np.pi / 180.0))  # N·m
    exp_med = np.median(t_expected[is_pass])
    exp_p99 = np.quantile(t_expected[is_pass], 0.99)
    tgt_med = (
        np.median(pass_t_target[is_pass])
        if pass_t_target.size == len(mode)
        else float("nan")
    )
    tgt_p99 = (
        np.quantile(pass_t_target[is_pass], 0.99)
        if pass_t_target.size == len(mode)
        else float("nan")
    )
else:
    exp_med = exp_p99 = tgt_med = tgt_p99 = float("nan")

# Saturation check in PASS
if wm_raw.size == len(mode) and pass_n > 0:
    pass_wm = wm_raw[is_pass]
    sat_steps = int((pass_wm >= 0.95).sum())
    sat_max = float(pass_wm.max())
else:
    sat_steps = 0
    sat_max = float("nan")

print(f"PASS steps: {pass_n}")
print(f"PASS T_expected median/99%: {exp_med:.3e} {exp_p99:.3e}")
print(f"PASS T_target  median/99%: {tgt_med:.3e} {tgt_p99:.3e}")
print(f"PASS saturation steps (wm_raw>=0.95): {sat_steps} (max wm_raw={sat_max:.3f})")
print(
    "PASS torque OK?",
    (abs(tgt_med - exp_med) < 1e-5) and (abs(tgt_p99 - exp_p99) < 1e-4),
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

mode = np.array(ditl_case.mode)
wm_raw = np.array(getattr(ditl_case, "wheel_momentum_fraction_raw", []), dtype=float)

step = float(ditl_case.step_size)
t_hr = np.arange(len(mode)) * step / 3600.0

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(t_hr, wm_raw, label="Max wheel momentum frac (raw)", color="C0")

# Shade modes
mode_colors = {
    ACSMode.SCIENCE: "#d9f2d9",
    ACSMode.SLEWING: "#ffe0b3",
    ACSMode.PASS: "#d0e1ff",
    ACSMode.SAA: "#e0e0e0",
    ACSMode.CHARGING: "#f5d0d0",
    ACSMode.SAFE: "#f0b3b3",
}
legend_shown = set()
for i, m in enumerate(mode):
    color = mode_colors.get(m)
    if color is None:
        continue
    label = m.name if m not in legend_shown else None
    ax.axvspan(t_hr[i], t_hr[i] + step / 3600.0, color=color, alpha=0.25, label=label)
    legend_shown.add(m)

ax.set_xlabel("Time (hours)")
ax.set_ylabel("Momentum fraction")
ax.set_ylim(0, 1.05)
ax.grid(True, alpha=0.3)
ax.legend(loc="upper right", ncol=3, fontsize=8)
plt.show()

In [ ]:
mode

In [ ]:
import numpy as np

mode = np.array(ditl_case.mode)
step = float(ditl_case.step_size)
n = len(mode)

# Pull telemetry arrays (if present)
pass_rate = np.array(getattr(ditl_case, "pass_tracking_rate_deg_s", []), dtype=float)
pass_t_target = np.array(getattr(ditl_case, "pass_torque_target_mag", []), dtype=float)
pass_t_actual = np.array(getattr(ditl_case, "pass_torque_actual_mag", []), dtype=float)

# Estimate RA/Dec rate from recorded pointing
ra = np.array(ditl_case.ra, dtype=float)
dec = np.array(ditl_case.dec, dtype=float)
# unwrap RA for rate estimate
ra_unwrap = np.unwrap(np.deg2rad(ra))
dec_rad = np.deg2rad(dec)
dra = np.diff(ra_unwrap)
ddec = np.diff(dec_rad)
rate_est = np.rad2deg(np.sqrt(dra * dra + ddec * ddec) / step)

# Align arrays to diff length
mode_mid = mode[1:]  # diff aligns to step i->i+1
pass_mask = mode_mid == ACSMode.PASS

print("len(mode):", len(mode))
print("len(pass_rate):", len(pass_rate))
print("len(pass_t_target):", len(pass_t_target))
print("len(pass_t_actual):", len(pass_t_actual))
print("len(rate_est):", len(rate_est))

# PASS stats
if pass_mask.any():
    print(
        "PASS rate_est median/99%/max:",
        np.median(rate_est[pass_mask]),
        np.quantile(rate_est[pass_mask], 0.99),
        rate_est[pass_mask].max(),
    )
else:
    print("No PASS steps in rate_est")

if pass_rate.size:
    pm = mode == ACSMode.PASS
    print(
        "PASS pass_rate median/99%/max:",
        np.median(pass_rate[pm]) if pm.any() else None,
        np.quantile(pass_rate[pm], 0.99) if pm.any() else None,
        pass_rate[pm].max() if pm.any() else None,
    )
else:
    print("pass_tracking_rate_deg_s not recorded")

# Spot-check PASS rows where pass telemetry is zero but rate_est is not
if pass_rate.size:
    pm = mode == ACSMode.PASS
    idxs = np.where(pm & (pass_rate == 0.0))[0][:10]
    print("PASS steps with pass_rate==0 (first 10):", idxs)
    for i in idxs:
        if i < len(rate_est):
            print(
                f"i={i} t={i * step / 3600:.3f}h mode={mode[i].name} "
                f"rate_est={rate_est[i]:.3f} deg/s "
                f"Tpass={pass_t_target[i]:.3e} Tact={pass_t_actual[i]:.3e}"
            )

In [ ]:
acs = ditl_case.acs
pm = np.where(mode == ACSMode.PASS)[0]
print("PASS steps:", pm.size)
if pm.size:
    # sample a few PASS indices
    for i in pm[:10]:
        print(
            f"i={i} t={i * step / 3600:.3f}h current_pass is None? {acs.current_pass is None}"
        )

In [ ]:
import numpy as np

mode = np.array(ditl_case.mode)
pass_rate = np.array(getattr(ditl_case, "pass_tracking_rate_deg_s", []), dtype=float)
tpass = np.array(getattr(ditl_case, "pass_torque_target_mag", []), dtype=float)

n = len(mode)
step = float(ditl_case.step_size)

# Align lengths
if pass_rate.size != n:
    print("pass_rate length mismatch:", pass_rate.size, "vs", n)
if tpass.size != n:
    print("pass_t_target length mismatch:", tpass.size, "vs", n)

# Compute alpha from pass_rate (deg/s -> rad/s)
rate_rad = np.deg2rad(pass_rate)
alpha = np.diff(rate_rad) / step  # rad/s^2, length n-1

# Use PASS interior steps for stable comparison
pass_interior = (
    (mode[:-2] == ACSMode.PASS)
    & (mode[1:-1] == ACSMode.PASS)
    & (mode[2:] == ACSMode.PASS)
)

# Map alpha to center step
alpha_mid = alpha[1:]  # align with mode[1:-1]
tpass_mid = tpass[1:-1]

# Spacecraft inertia along axis (use average for quick check)
i_axis = float(np.mean(ditl_case.config.spacecraft_bus.attitude_control.spacecraft_moi))
t_expected = i_axis * np.abs(alpha_mid)

mask = pass_interior & np.isfinite(t_expected) & np.isfinite(tpass_mid)

print("PASS interior samples:", mask.sum())
if mask.any():
    print(
        "PASS T_expected median/99%:",
        np.median(t_expected[mask]),
        np.quantile(t_expected[mask], 0.99),
    )
    print(
        "PASS T_target  median/99%:",
        np.median(np.abs(tpass_mid[mask])),
        np.quantile(np.abs(tpass_mid[mask]), 0.99),
    )

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

t_hours = (np.array(ditl_case.utime) - ditl_case.utime[0]) / 3600.0
total = np.array(ditl_case.disturbance_total, dtype=float)
gg = np.array(ditl_case.disturbance_gg, dtype=float)
drag = np.array(ditl_case.disturbance_drag, dtype=float)
srp = np.array(ditl_case.disturbance_srp, dtype=float)
mag = np.array(ditl_case.disturbance_mag, dtype=float)

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(t_hours, total, label="Total", color="C0")
ax.plot(t_hours, gg, label="Gravity gradient", color="C1", alpha=0.7)
ax.plot(t_hours, drag, label="Aero drag", color="C2", alpha=0.7)
ax.plot(t_hours, srp, label="Solar pressure", color="C3", alpha=0.7)
ax.plot(t_hours, mag, label="Residual magnetic", color="C4", alpha=0.7)
ax.set_yscale("log")
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Torque magnitude (N·m)")
ax.set_title("External disturbance torques over time")
ax.legend(loc="upper right", ncol=2)
ax.grid(alpha=0.2)
ax.set_ylim(1e-10, 1e-4)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

dt = float(ditl_case.step_size)
t_hours = (np.array(ditl_case.utime) - ditl_case.utime[0]) / 3600.0

# Disturbance torque magnitudes
gg = np.array(ditl_case.disturbance_gg, dtype=float)
drag = np.array(ditl_case.disturbance_drag, dtype=float)
srp = np.array(ditl_case.disturbance_srp, dtype=float)
mag = np.array(ditl_case.disturbance_mag, dtype=float)
dist_total = np.array(ditl_case.disturbance_total, dtype=float)

# MTQ projection (best-case unload) and wheel momentum fraction
mtq_proj = np.array(getattr(ditl_case, "mtq_proj_max", []), dtype=float)
wm_frac = np.array(getattr(ditl_case, "wheel_momentum_fraction_raw", []), dtype=float)

# Cumulative impulse from disturbances and MTQ
cum_gg = np.cumsum(gg * dt)
cum_drag = np.cumsum(drag * dt)
cum_srp = np.cumsum(srp * dt)
cum_mag = np.cumsum(mag * dt)
cum_mtq = np.cumsum(mtq_proj * dt)  # optimistic: assumes perfect opposing direction

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Momentum fraction over time
ax1.plot(t_hours, wm_frac, label="Max wheel momentum fraction (raw)", color="C0")
ax1.set_ylabel("Momentum fraction")
ax1.set_title("Wheel momentum vs external torques/MTQ unload")
ax1.grid(alpha=0.2)
ax1.legend(loc="lower right")

# Cumulative impulse comparison
ax2.plot(t_hours, cum_gg, label="GG impulse", color="C1")
ax2.plot(t_hours, cum_drag, label="Drag impulse", color="C2")
ax2.plot(t_hours, cum_srp, label="SRP impulse", color="C3")
ax2.plot(t_hours, cum_mag, label="Residual mag impulse", color="C4")
ax2.plot(
    t_hours,
    np.cumsum(dist_total * dt),
    label="Total disturbance impulse",
    color="C5",
    linestyle="--",
)
ax2.plot(t_hours, cum_mtq, label="MTQ projected impulse (best-case oppose)", color="C6")
ax2.set_xlabel("Time (hours)")
ax2.set_ylabel("Impulse (N·m·s)")
ax2.grid(alpha=0.2)
ax2.set_yscale("log")
ax2.set_ylim(bottom=1e-21)
ax2.legend(loc="lower right", ncol=2)

plt.tight_layout()
plt.show()

In [ ]:
wheel_grid = [
    ("1.0Nms_0.06Nm", 1.0, 0.06),
    ("2.0Nms_0.12Nm", 2.0, 0.12),
    ("1.2Nms_0.25Nm", 1.2, 0.25),
    ("4.0Nms_0.25Nm", 4.0, 0.25),
    ("2.5Nms_0.50Nm", 2.5, 0.50),
    ("8.0Nms_0.25Nm", 8.0, 0.25),
    ("5.0Nms_0.50Nm", 5.0, 0.50),
]
mtq_grid = [32, 50, 100]

results = []
ditls = {}
for mtq in mtq_grid:
    for name, mom, torq in wheel_grid:
        key = f"{name}_mtq{mtq}"
        ditl_case, summary = run_case(name, mom, torq, mtq)
        ditls[key] = ditl_case
        summary["key"] = key
        results.append(summary)

df = pd.DataFrame(results).set_index("key")
df_sorted = df.sort_values(
    by=["science_%", "slew_%", "desat_events", "max_mom_frac", "max_torque_frac"],
    ascending=[False, True, True, True, True],
)
display(df_sorted.head())

# Plot the top-ranked case
case_to_plot = df_sorted.index[0]
fig, axes = plot_ditl_telemetry(
    ditls[case_to_plot],
    figsize=(12, 8),
    config=VisualizationConfig(font_family="DejaVu Sans"),
)
fig.suptitle(f"Case: {case_to_plot}")
plt.show()

In [ ]:
import numpy as np

from conops.common import ACSCommandType, ACSMode

# Timebase
t = np.array(ditl_case.utime)
dt = np.diff(t, prepend=t[0])
hrs = dt / 3600

m = np.array(ditl_case.mode)
time_by_mode = {
    "SCIENCE": np.sum(hrs[m == ACSMode.SCIENCE]),
    "SLEWING": np.sum(hrs[m == ACSMode.SLEWING]),
    "PASS": np.sum(hrs[m == ACSMode.PASS]),
    "SAFE": np.sum(hrs[m == ACSMode.SAFE]),
    "CHARGING": np.sum(hrs[m == ACSMode.CHARGING]),
    "SAA": np.sum(hrs[m == ACSMode.SAA]),
}
print("Hours by mode:", time_by_mode)

# Desat
desat_steps = getattr(ditl_case, "desat_time_steps", 0)
print(f"Desat hours: {desat_steps * ditl_case.step_size / 3600:.2f}")

# Wheel margins
wm = np.array(getattr(ditl_case, "wheel_momentum_fraction", []))
wt = np.array(getattr(ditl_case, "wheel_torque_fraction", []))
if wm.size:
    print(f"Wheel momentum: max={wm.max():.2f}, p95={np.quantile(wm, 0.95):.2f}")
if wt.size:
    print(f"Wheel torque: max={wt.max():.2f}, p95={np.quantile(wt, 0.95):.2f}")

# Slew outcomes
slews = [
    c
    for c in ditl_case.acs.executed_commands
    if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew
]
dists = [c.slew.slewdist for c in slews]
rejected = getattr(ditl_case.acs, "headroom_rejects", 0)
print(
    f"Slews: {len(slews)} executed, {rejected} rejected; dist mean={np.mean(dists):.1f} deg"
)

# Simple histograms (numpy bins)
hist_mom, bins_m = np.histogram(wm, bins=np.linspace(0, 1, 11)) if wm.size else ([], [])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

time_hours = (np.array(ditl_case.utime) - ditl_case.utime[0]) / 3600

plt.figure(figsize=(10, 5))
# plt.plot(time_hours, ditl_case.disturbance_total, label="Total")
plt.plot(time_hours, ditl_case.disturbance_gg, label="Gravity gradient")
plt.plot(time_hours, ditl_case.disturbance_drag, label="Aero drag")
plt.plot(time_hours, ditl_case.disturbance_srp, label="Solar pressure")
plt.plot(time_hours, ditl_case.disturbance_mag, label="Magnetic (residual)")
plt.xlabel("Time (hours)")
plt.ylabel("Torque magnitude (N·m)")
plt.legend()
plt.yscale("log")
plt.tight_layout()
plt.show()

In [ ]:
from datetime import datetime, timezone

import numpy as np

# enable MSIS
ditl_case.config.spacecraft_bus.attitude_control.use_msis_density = True

# after your DITL calc:
acs = ditl_case.acs
ephem_local = ditl_case.ephem

densities = []
alts_km = []
for ut in ditl_case.utime:
    idx = ephem_local.index(datetime.fromtimestamp(ut, tz=timezone.utc))
    r_vec = np.array(ephem_local.gcrs_pv.position[idx])
    # normalize to meters if in km
    if np.linalg.norm(r_vec) < 1e6:
        r_vec = r_vec * 1000.0
    alt_m = max(0.0, np.linalg.norm(r_vec) - 6378e3)
    lat = (
        float(getattr(ephem_local, "lat", [0])[idx])
        if hasattr(ephem_local, "lat")
        else 0.0
    )
    lon = (
        float(getattr(ephem_local, "long", [0])[idx])
        if hasattr(ephem_local, "long")
        else 0.0
    )
    rho = acs._atmospheric_density(ut, alt_m, lat, lon)
    densities.append(rho)
    alts_km.append(alt_m / 1000.0)

# Plot density vs time
import matplotlib.pyplot as plt

time_hours = (np.array(ditl_case.utime) - ditl_case.utime[0]) / 3600
plt.semilogy(time_hours, densities)
plt.xlabel("Time (hours)")
plt.ylabel("Density (kg/m^3)")
plt.tight_layout()
plt.show()

In [ ]:
acs = ditl_case.acs
acs._use_msis = True  # ensure MSIS path
ut0 = ditl_case.utime[0]
idx = ditl_case.ephem.index(datetime.fromtimestamp(ut0, tz=timezone.utc))
r = np.array(ditl_case.ephem.gcrs_pv.position[idx])
if np.linalg.norm(r) < 1e6:
    r = r * 1000
alt = np.linalg.norm(r) - 6378e3
lat = float(getattr(ditl_case.ephem, "lat", [0])[idx])
lon = float(getattr(ditl_case.ephem, "long", [0])[idx])
rho = acs._atmospheric_density(ut0, alt, lat, lon)
print(f"alt={alt / 1000:.1f} km rho={rho:.3e} kg/m^3")

In [ ]:
alt

In [ ]:
ditl_case.print_statistics()

In [ ]:
from conops.common import ACSCommandType, ACSMode

time_h = (np.array(ditl_case.utime) - ditl_case.utime[0]) / 3600
slew_cmds = [
    c
    for c in ditl_case.acs.executed_commands
    if c.command_type == ACSCommandType.SLEW_TO_TARGET
]
desat_cmds = [
    c for c in ditl_case.acs.executed_commands if c.command_type == ACSCommandType.DESAT
]
print(
    "DESAT windows:",
    [(c.execution_time, getattr(c, "duration", None)) for c in desat_cmds],
)

# Overlay where mode==SLEWING to see it spans settle/desat
import matplotlib.pyplot as plt

m = np.array(ditl_case.mode)
plt.plot(time_h, ditl_case.ra)
plt.scatter(
    time_h[m == ACSMode.SLEWING],
    np.array(ditl_case.ra)[m == ACSMode.SLEWING],
    s=8,
    color="r",
)

In [ ]:
from conops.common import ACSCommandType

cmds = [
    c
    for c in ditl_case.acs.executed_commands
    if c.command_type == ACSCommandType.SLEW_TO_TARGET
]
for c in cmds:
    dist = c.slew.slewdist  # deg along great circle
    t = c.slew.slewtime  # s (includes settle)
    avg_rate = dist / t if t else 0
    a = getattr(c.slew, "_accel_override", None)
    vmax = getattr(c.slew, "_vmax_override", None)
    print(
        f"obsid={c.slew.obsid} dist={dist:.2f}deg time={t:.1f}s avg={avg_rate:.3f} deg/s accel={a} vmax={vmax}"
    )

In [ ]:
import numpy as np

from conops.common import separation

# great-circle step-to-step rate
rates = []
for i in range(1, len(ditl_case.ra)):
    r = separation(
        [np.deg2rad(ditl_case.ra[i - 1]), np.deg2rad(ditl_case.dec[i - 1])],
        [np.deg2rad(ditl_case.ra[i]), np.deg2rad(ditl_case.dec[i])],
    )  # deg
    d = np.rad2deg(r)
    rates.append(d / ditl_case.step_size)  # deg/s
print("Max step rate:", max(rates), "deg/s")

In [ ]:
import numpy as np

from conops.common import ACSCommandType

dists = [
    c.slew.slewdist
    for c in ditl_case.acs.executed_commands
    if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew
]
print("mean dist:", np.mean(dists), "deg", "max:", np.max(dists), "deg")

In [ ]:
import numpy as np

from conops.common import ACSCommandType, separation

# Slew distances actually flown
dists = [
    c.slew.slewdist
    for c in ditl_case.acs.executed_commands
    if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew
]
print(
    f"Count={len(dists)}, mean={np.mean(dists):.1f} deg, median={np.median(dists):.1f}, max={np.max(dists):.1f}"
)

# Time in SLEWING
mode_array = np.array(ditl_case.mode)
frac_slewing = np.mean(mode_array == 1) * 100
print(f"SLEWING fraction: {frac_slewing:.1f}%")

# Effective accel/rate per slew
for c in ditl_case.acs.executed_commands:
    if c.command_type == ACSCommandType.SLEW_TO_TARGET and c.slew:
        print(
            f"obsid={c.slew.obsid} dist={c.slew.slewdist:.1f} deg, "
            f"accel={getattr(c.slew, '_accel_override', None)}, "
            f"vmax={getattr(c.slew, '_vmax_override', None)}"
        )

In [ ]:
import numpy as np

from conops.common import ACSMode

mode_arr = np.array(ditl_case.mode)
science_frac = np.mean(mode_arr == ACSMode.SCIENCE) * 100
slew_frac = np.mean(mode_arr == ACSMode.SLEWING) * 100
print(f"SCIENCE: {science_frac:.1f}%")
print(f"SLEWING: {slew_frac:.1f}%")